In [1]:
#!conda install beautifulsoup4

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import csv

# Part 1

In [2]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'lxml')

postCode = []
boroughName = []
neighborhoodName = []

### Extracting data, cleaning it and presenting it in a DF

In [3]:
for row in soup.find('table').find_all('tr'):  #get table and table rows
    cells = row.find_all('td')
    if(len(cells) > 0):  #i.e. not empty
        postCode.append(cells[0].text.rstrip('\n'))
        boroughName.append(cells[1].text.rstrip('\n'))
        neighborhoodName.append(cells[2].text.rstrip('\n'))  #removing new line character

In [4]:
df = pd.DataFrame({"Postal Code": postCode,"Borough": boroughName,"Neighborhood": neighborhoodName}) #create df

df2 = df[df.Borough != 'Not assigned'].reset_index(drop=True) #drop unassigned

df3 = df2.groupby(['Postal Code','Borough'], as_index=False).agg(lambda x: ', '.join(x)) #join neighbourhoods of same borough

df3.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Cleaned DF now has 103 rows

In [5]:
df3.shape

(103, 3)

# Part 2

In [6]:
import csv
import os
os.chdir('/Users/Nikita/Desktop/IBM course') 
csv_file = open('Geospatial_Coordinates.csv', 'r') 
#use given csv file provided by IBM

In [7]:
GeoCoords = pd.read_csv(csv_file) 
df3 = pd.merge(df3, GeoCoords, how= 'inner', on = 'Postal Code') #use given csv file provided by IBM
    
df3.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [8]:
csv_file.close()

# Part 3

In [9]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

In [10]:
# get coords of Etobicoke
address = 'Etobicoke , Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [11]:
Etobicoke_data = df3[df3['Borough'] == 'Etobicoke'].reset_index(drop=True)
Etobicoke_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
3,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
4,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999


In [12]:
address = 'Etobicoke, Toronto'

geolocator = Nominatim(user_agent="Etobicoke_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_Etobicoke = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(Etobicoke_data['Latitude'], Etobicoke_data['Longitude'], Etobicoke_data['Borough'], Etobicoke_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Etobicoke)  
    
map_Etobicoke

In [13]:
CLIENT_ID = "hidden"# your Foursquare ID
CLIENT_SECRET = 'hidden' # your Foursquare Secret
VERSION = 'hidden' # Foursquare API version

LIMIT = 100 
radius = 500 


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:

Etobicoke_venues = getNearbyVenues(names=Etobicoke_data['Neighborhood'],
                                   latitudes=Etobicoke_data['Latitude'],
                                   longitudes=Etobicoke_data['Longitude']
                                  )
Etobicoke_venues.head()

New Toronto, Mimico South, Humber Bay Shores
Alderwood, Long Branch
The Kingsway, Montgomery Road, Old Mill North
Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East
Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West
Islington Avenue, Humber Valley Village
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Westmount
Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens
South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens
Northwest, West Humber - Clairville


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,LCBO,43.602281,-79.499302,Liquor Store
1,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Domino's Pizza,43.601583,-79.500905,Pizza Place
2,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,New Toronto Fish & Chips,43.601849,-79.503281,Restaurant
3,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Delicia Bakery & Pastry,43.601403,-79.503012,Bakery
4,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Lucky Dice Restaurant,43.601392,-79.503056,Café


In [15]:
Etobicoke_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alderwood, Long Branch",7,7,7,7,7,7
"Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood",10,10,10,10,10,10
"Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens",3,3,3,3,3,3
"Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West",14,14,14,14,14,14
"New Toronto, Mimico South, Humber Bay Shores",13,13,13,13,13,13
"Northwest, West Humber - Clairville",4,4,4,4,4,4
"Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East",2,2,2,2,2,2
"South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens",9,9,9,9,9,9
"The Kingsway, Montgomery Road, Old Mill North",1,1,1,1,1,1


In [16]:
print('There are {} uniques categories.'.format(len(Etobicoke_venues['Venue Category'].unique())))

There are 40 uniques categories.


In [17]:
# one hot encoding
Etobicoke_onehot = pd.get_dummies(Etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Etobicoke_onehot['Neighborhood'] = Etobicoke_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Etobicoke_onehot.columns[-1]] + list(Etobicoke_onehot.columns[:-1])
Etobicoke_onehot = Etobicoke_onehot[fixed_columns]

Etobicoke_onehot.head()

,Neighborhood,American Restaurant,Bakery,Bar,Baseball Field,Beer Store,Burger Joint,Burrito Place,Café,Chinese Restaurant,...,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Seafood Restaurant,Skating Rink,Supplement Shop,Tanning Salon,Wings Joint
0,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,"New Toronto, Mimico South, Humber Bay Shores",0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
Etobicoke_grouped = Etobicoke_onehot.groupby('Neighborhood').mean().reset_index()
Etobicoke_grouped

,Neighborhood,American Restaurant,Bakery,Bar,Baseball Field,Beer Store,Burger Joint,Burrito Place,Café,Chinese Restaurant,...,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Seafood Restaurant,Skating Rink,Supplement Shop,Tanning Salon,Wings Joint
0,"Alderwood, Long Branch",0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.142857,0.00,0.000000,0.0,0.142857,0.000000,0.142857,0.000000,0.000000,0.000000
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.000000,0.000000,0.00,0.0,0.100000,0.000000,0.000000,0.100000,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Kingsview Village, St. Phillips, Martin Grove ...",0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Mimico NW, The Queensway West, South of Bloor,...",0.000000,0.071429,0.00,0.0,0.000000,0.071429,0.071429,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.071429,0.000000,0.000000,0.071429,0.071429,0.071429
4,"New Toronto, Mimico South, Humber Bay Shores",0.076923,0.076923,0.00,0.0,0.000000,0.000000,0.000000,0.076923,0.000000,...,0.000000,0.00,0.076923,0.0,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000
5,"Northwest, West Humber - Clairville",0.000000,0.000000,0.25,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.25,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Old Mill South, King's Mill Park, Sunnylea, Hu...",0.000000,0.000000,0.00,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"South Steeles, Silverstone, Humbergate, Jamest...",0.000000,0.000000,0.00,0.0,0.111111,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000
8,"The Kingsway, Montgomery Road, Old Mill North",0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"West Deane Park, Princess Gardens, Martin Grov...",0.000000,1.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [19]:
num_top_venues = 5

for hood in Etobicoke_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Etobicoke_grouped[Etobicoke_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1             Gym  0.14
2    Skating Rink  0.14
3  Sandwich Place  0.14
4     Coffee Shop  0.14


----Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood----
        venue  freq
0        Park   0.2
1  Beer Store   0.1
2   Pet Store   0.1
3        Café   0.1
4    Pharmacy   0.1


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                 venue  freq
0    Mobile Phone Shop  0.33
1       Sandwich Place  0.33
2                 Park  0.33
3  American Restaurant  0.00
4          Pizza Place  0.00


----Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West----
                  venue  freq
0           Wings Joint  0.07
1         Grocery Store  0.07
2        Hardware Store  0.07
3           Flower Shop  0.07
4  Fast Food Restaurant  0.07


----New Toronto, Mimico South, Humber Bay Shores----
                 venue  freq
0  Ameri

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Etobicoke_grouped['Neighborhood']

for ind in np.arange(Etobicoke_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Etobicoke_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Sandwich Place,Pub,Coffee Shop,Chinese Restaurant,Fried Chicken Joint,Flower Shop,Fast Food Restaurant
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Park,Pharmacy,Beer Store,Liquor Store,Café,Convenience Store,Pizza Place,Coffee Shop,Pet Store,Flower Shop
2,"Kingsview Village, St. Phillips, Martin Grove ...",Sandwich Place,Mobile Phone Shop,Park,Wings Joint,Coffee Shop,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store
3,"Mimico NW, The Queensway West, South of Bloor,...",Wings Joint,Tanning Salon,Bakery,Burger Joint,Burrito Place,Convenience Store,Discount Store,Fast Food Restaurant,Flower Shop,Grocery Store
4,"New Toronto, Mimico South, Humber Bay Shores",American Restaurant,Pizza Place,Fast Food Restaurant,Gym,Liquor Store,Mexican Restaurant,Coffee Shop,Café,Pharmacy,Pet Store


In [22]:
#Run k-means to cluster the neighborhood into 5 clusters
kclusters = 5
Etobicoke_grouped_clustering = Etobicoke_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Etobicoke_grouped_clustering)
kmeans.labels_[0:10]

array([0, 0, 4, 0, 0, 0, 3, 0, 1, 2])

In [23]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Etobicoke_merged = Etobicoke_data
Etobicoke_merged = Etobicoke_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Etobicoke_merged.head() 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,0.0,American Restaurant,Pizza Place,Fast Food Restaurant,Gym,Liquor Store,Mexican Restaurant,Coffee Shop,Café,Pharmacy,Pet Store
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,0.0,Pizza Place,Gym,Skating Rink,Sandwich Place,Pub,Coffee Shop,Chinese Restaurant,Fried Chicken Joint,Flower Shop,Fast Food Restaurant
2,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,1.0,River,Wings Joint,Coffee Shop,Garden Center,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store
3,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,3.0,Home Service,Baseball Field,Convenience Store,Grocery Store,Garden Center,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store
4,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,0.0,Wings Joint,Tanning Salon,Bakery,Burger Joint,Burrito Place,Convenience Store,Discount Store,Fast Food Restaurant,Flower Shop,Grocery Store


In [24]:

# create map
map_clusterd = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Etobicoke_merged['Latitude'], Etobicoke_merged['Longitude'],
                                  Etobicoke_merged['Neighborhood'], [0,1,2,3,4]):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusterd)
       
map_clusterd

In [25]:

Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 0, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Etobicoke,0.0,American Restaurant,Pizza Place,Fast Food Restaurant,Gym,Liquor Store,Mexican Restaurant,Coffee Shop,Café,Pharmacy,Pet Store
1,Etobicoke,0.0,Pizza Place,Gym,Skating Rink,Sandwich Place,Pub,Coffee Shop,Chinese Restaurant,Fried Chicken Joint,Flower Shop,Fast Food Restaurant
4,Etobicoke,0.0,Wings Joint,Tanning Salon,Bakery,Burger Joint,Burrito Place,Convenience Store,Discount Store,Fast Food Restaurant,Flower Shop,Grocery Store
7,Etobicoke,0.0,Park,Pharmacy,Beer Store,Liquor Store,Café,Convenience Store,Pizza Place,Coffee Shop,Pet Store,Flower Shop
8,Etobicoke,0.0,Chinese Restaurant,Coffee Shop,Intersection,Sandwich Place,Middle Eastern Restaurant,Discount Store,Pizza Place,Fried Chicken Joint,Flower Shop,Fast Food Restaurant
10,Etobicoke,0.0,Grocery Store,Pizza Place,Coffee Shop,Fried Chicken Joint,Fast Food Restaurant,Sandwich Place,Beer Store,Pharmacy,Flower Shop,Drugstore
11,Etobicoke,0.0,Garden Center,Bar,Drugstore,Rental Car Location,Wings Joint,Coffee Shop,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Discount Store


In [26]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 1, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Etobicoke,1.0,River,Wings Joint,Coffee Shop,Garden Center,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store


In [27]:

Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 2, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Etobicoke,2.0,Bakery,Wings Joint,Convenience Store,Grocery Store,Garden Center,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store


In [28]:

Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 3, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Etobicoke,3.0,Home Service,Baseball Field,Convenience Store,Grocery Store,Garden Center,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store


In [29]:
Etobicoke_merged.loc[Etobicoke_merged['Cluster Labels'] == 4, Etobicoke_merged.columns[[1] + list(range(5, Etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Etobicoke,4.0,Sandwich Place,Mobile Phone Shop,Park,Wings Joint,Coffee Shop,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store
